In [1]:
import os
import sys
import collections

In [2]:
sys.path.extend(['..'])

In [3]:
sys.path

['/home/b/jchun/.pyenv/versions/3.6.8/lib/python36.zip',
 '/home/b/jchun/.pyenv/versions/3.6.8/lib/python3.6',
 '/home/b/jchun/.pyenv/versions/3.6.8/lib/python3.6/lib-dynload',
 '',
 '/home/b/jchun/Documents/Envs/Lycastus-Py3.6.8-GPU/lib/python3.6/site-packages',
 '/home/b/jchun/Documents/Envs/Lycastus-Py3.6.8-GPU/lib/python3.6/site-packages/IPython/extensions',
 '/home/b/jchun/.ipython',
 '..']

In [4]:
import numpy as np
import tensorflow as tf

In [5]:
import drs
import modeling
from run_classifier import convert_single_example, model_fn_builder, InputExample
import tokenization
import attention

In [6]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [7]:
# flags
model_dir  = "../drs/output_viz_implicit_uncased_1"
label_list = drs.get_senses_with_degree(2)

In [8]:
# bert paths
bert_config_file = "../uncased_L-12_H-768_A-12/bert_config.json"
bert_init_ckpt   = "../uncased_L-12_H-768_A-12/bert_model.ckpt"
bert_vocab_file  = "../uncased_L-12_H-768_A-12/vocab.txt"

In [9]:
# experiment settings
seq_length = 128
num_hidden_layers=12
num_attention_heads=12

In [10]:
# tokenizer
tokenizer  = tokenization.FullTokenizer(
      vocab_file=bert_vocab_file,
      do_lower_case=True)

In [11]:
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

In [12]:
os.path.exists(model_dir)

True

In [13]:
label_list

['Contingency.Condition',
 'Expansion.Alternative',
 'Temporal.Asynchronous',
 'Comparison.Contrast',
 'Expansion.Instantiation',
 'Expansion.Exception',
 'Comparison.Concession',
 'Expansion.Conjunction',
 'Comparison',
 'Temporal',
 'Expansion.Restatement',
 'EntRel',
 'Expansion',
 'Contingency',
 'Temporal.Synchrony',
 'Contingency.Cause']

In [14]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=len(label_list),
      init_checkpoint=bert_init_ckpt,
      use_one_hot_embeddings=False,
      learning_rate=None,
      num_train_steps=None,
      num_warmup_steps=None)

In [15]:
run_config = tf.contrib.tpu.RunConfig(
      cluster=None,
      model_dir=model_dir)

In [16]:
model = tf.contrib.tpu.TPUEstimator(
    use_tpu=False,
    model_fn=model_fn,
    config=run_config)

INFO:tensorflow:Using config: {'_model_dir': '../drs/output_viz_implicit_uncased_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f34c8f8ed68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_

In [17]:
# model.get_variable_names()

In [18]:
def input_fn_builder(input_):
    name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
      }
    
    def input_fn(params):
        return tf.parse_single_example(input_, name_to_features)
    return input_fn

In [19]:
text_a = "I went too"

In [20]:
text_b = "I knew little of where, however."

In [21]:
input_example = InputExample(guid=0, text_a="I went too", text_b="I knew little of where, however.", label='EntRel')

In [22]:
feature = convert_single_example(0, input_example, label_list, seq_length, tokenizer)

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 0
INFO:tensorflow:tokens: [CLS] i went too [SEP] i knew little of where , however . [SEP]
INFO:tensorflow:input_ids: 101 1045 2253 2205 102 1045 2354 2210 1997 2073 1010 2174 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [23]:
sum(feature.input_mask)

14

In [24]:
def create_int_feature(values):
    f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
    return f

In [25]:
features = collections.OrderedDict()
features["input_ids"] = create_int_feature(feature.input_ids)
features["input_mask"] = create_int_feature(feature.input_mask)
features["segment_ids"] = create_int_feature(feature.segment_ids)
features["label_ids"] = create_int_feature([feature.label_id])
features["is_real_example"] = create_int_feature(
    [int(feature.is_real_example)])

In [26]:
tf_example = tf.train.Example(features=tf.train.Features(feature=features))

In [27]:
viz_pred_file = os.path.join(model_dir, "viz_pred.tf_record")

In [28]:
writer = tf.python_io.TFRecordWriter(viz_pred_file)
writer.write(tf_example.SerializeToString())
writer.close()

In [29]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""
    
    name_to_features = {
          "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
          "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
          "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
          "label_ids": tf.FixedLenFeature([], tf.int64),
          "is_real_example": tf.FixedLenFeature([], tf.int64),
        }

    def input_fn(params):
        """The actual input function."""
        d = tf.data.TFRecordDataset(input_file)
        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: tf.parse_single_example(record, name_to_features),
                batch_size=1,
                drop_remainder=drop_remainder))
        return d

    return input_fn

In [30]:
predict_input_fn = file_based_input_fn_builder(
        input_file=viz_pred_file,
        seq_length=seq_length,
        is_training=False,
        drop_remainder=False)

In [31]:
result = model.predict(input_fn=predict_input_fn)

In [32]:
for prediction in result:
    probabilities = prediction["probabilities"]
    label = label_list[np.argmax(probabilities)]
    viz_attns = prediction['viz_attns']

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name =

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKP

INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/

In [33]:
label

'EntRel'

In [34]:
viz_attns.shape

(12, 12, 128, 128)

# Now for Viz

In [35]:
__location__ = os.path.realpath(
    os.path.join(os.getcwd(), os.path.dirname("__file__")))
vis_js = open(os.path.join(__location__, 'attention.js')).read()

In [36]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
  }
});

<IPython.core.display.Javascript object>

In [37]:
tokens_a = tokenizer.tokenize(text_a)
tokens_b = tokenizer.tokenize(text_b)

In [38]:
tokens_a_delim = ['[CLS]'] + tokens_a + ['[SEP]']
tokens_b_delim = tokens_b + ['[SEP]']

In [39]:
viz_attns.shape

(12, 12, 128, 128)

In [40]:
for i in range(128):
    c = viz_attns[11][11][i]
    print()
    print(sum(c[:14]))
    print(c[:15])


0.9999999478459358
[0.00295094 0.0056231  0.02074752 0.00194957 0.2963441  0.00828897
 0.00620899 0.00579546 0.00145816 0.0020315  0.06707279 0.00093173
 0.26615116 0.31444594 0.        ]

0.999999966006726
[0.00350643 0.0169111  0.00889428 0.00127754 0.2857357  0.01416151
 0.00799289 0.01681064 0.00177513 0.00501807 0.07547547 0.00179576
 0.26169413 0.2989513  0.        ]

0.9999999166466296
[0.00511032 0.00766204 0.04963357 0.00222281 0.27983004 0.00708607
 0.00910446 0.01053133 0.00501973 0.0114472  0.06608088 0.00273531
 0.2517866  0.29174957 0.        ]

0.9999999560677679
[7.5871957e-04 1.9172188e-03 4.9682488e-03 5.3598377e-04 3.1267577e-01
 1.6933901e-03 1.0090694e-03 1.9553560e-03 2.3685647e-04 3.8068352e-04
 6.2080167e-02 4.7741033e-04 2.8149965e-01 3.2981142e-01 0.0000000e+00]

1.0000000170548446
[0.00063284 0.00359286 0.01061158 0.00102954 0.30539337 0.00231068
 0.00156911 0.00317549 0.00099512 0.00110633 0.06837295 0.00098182
 0.2765044  0.32372394 0.        ]

1.00000003

In [41]:
for i in range(12):
    layer_attn = viz_attns[i]
    print(layer_attn.shape)
    
    c = layer_attn[0, 2]
    print(sum(c))

(12, 128, 128)
1.000000029336661
(12, 128, 128)
0.9999999957508408
(12, 128, 128)
1.0000000877214708
(12, 128, 128)
0.9999999858800948
(12, 128, 128)
1.000000026077032
(12, 128, 128)
1.0000000110303517
(12, 128, 128)
1.0000000395702955
(12, 128, 128)
0.9999999854626367
(12, 128, 128)
1.0000000090076355
(12, 128, 128)
1.0000000398576958
(12, 128, 128)
1.0000000637955964
(12, 128, 128)
0.9999999391147867


In [42]:
new_attn = viz_attns[:,:,:14, :14]

In [43]:
new_attn.shape

(12, 12, 14, 14)

In [44]:
new_attn

array([[[[6.99609518e-02, 5.81403263e-02, 3.23766060e-02, ...,
          8.61096829e-02, 8.62188190e-02, 1.18510701e-01],
         [2.70160586e-01, 2.57041246e-01, 2.40734428e-01, ...,
          2.35638209e-03, 2.59424513e-03, 2.48519145e-03],
         [4.53961827e-02, 3.07193607e-01, 2.59493947e-01, ...,
          3.08130449e-03, 2.39554117e-03, 4.29130066e-03],
         ...,
         [6.03858717e-02, 4.15303744e-03, 5.73622715e-03, ...,
          5.33852167e-02, 6.07364587e-02, 1.07940800e-01],
         [5.38705476e-02, 1.40978966e-03, 1.07578025e-03, ...,
          6.13329522e-02, 8.58174264e-02, 2.05262721e-01],
         [3.23667675e-02, 8.96028243e-04, 3.76896904e-04, ...,
          9.29502621e-02, 1.02703542e-01, 1.72715947e-01]],

        [[4.36039031e-01, 7.90082850e-03, 4.30575758e-03, ...,
          3.40365805e-03, 1.48901213e-02, 3.20291612e-03],
         [1.65045857e-02, 3.59941833e-02, 2.95041472e-01, ...,
          8.11827108e-02, 2.95515941e-03, 5.13733225e-03],
        

In [45]:
new_attn[0][0][12].sum()

1.0

In [46]:
tokens_a_delim + tokens_b_delim

['[CLS]',
 'i',
 'went',
 'too',
 '[SEP]',
 'i',
 'knew',
 'little',
 'of',
 'where',
 ',',
 'however',
 '.',
 '[SEP]']

In [47]:
attention.show(tokens_a_delim, tokens_b_delim, viz_attns)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
got_attn = attention._get_attention(tokens_a_delim, tokens_b_delim, viz_attns)

AttributeError: module 'attention' has no attribute '_get_attention'

In [ ]:
ba = got_attn['ba']['att']

In [ ]:
import numpy as np

In [ ]:
ba_np = np.asarray(ba)

In [ ]:
ba_np.shape

In [ ]:
ba[0][0]

In [50]:
len(tokens_a_delim)

5

In [51]:
len(tokens_b_delim)

9